# Variance-level plots

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# plt.rcParams["lines.color"] = "w"
# plt.rcParams["text.color"] = "w"
# plt.rcParams["axes.labelcolor"] = "w"
# plt.rcParams["xtick.color"] = "w"
# plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
ls = [6, 7, 8, 9]

In [5]:
from gpuocean.utils import DoubleJetCase

args_list = []
init_list = []

for l in ls:
    doubleJetCase = DoubleJetCase.DoubleJetCase(gpu_ctx, DoubleJetCase.DoubleJetPerturbationType.SteadyState, ny=2**l, nx=2**(l+1))
    doubleJetCase_args, doubleJetCase_init, _ = doubleJetCase.getInitConditions()

    args_list.append(doubleJetCase_args)
    init_list.append(doubleJetCase_init)

In [6]:
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-26T11_10_32"
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-25T13_55_37"
source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-10-27T17_43_20"

In [7]:
vars = np.load(source_path+"/vars_"+str(10*24*3600)+".npy")
diff_vars = np.load(source_path+"/diff_vars_"+str(10*24*3600)+".npy")

In [8]:
vars, diff_vars

(array([[8.3266455e+03, 2.0601485e+08, 1.9769155e+08],
        [1.9638676e+04, 6.9546618e+08, 5.1919053e+08],
        [3.2229025e+04, 1.1429027e+09, 9.0254490e+08],
        [4.2792047e+04, 1.4441010e+09, 1.1609929e+09]], dtype=float32),
 array([[9.8156191e+03, 3.9142342e+08, 2.5070651e+08],
        [1.4287225e+04, 3.6043485e+08, 2.7581728e+08],
        [1.5784937e+04, 2.1321568e+08, 1.7720861e+08]], dtype=float32))

In [9]:
work_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/PracticalCost/2023-09-18T12_54_47"

In [10]:
works = np.mean(np.load(work_path+"/costsPure.npy"), axis=-1)
diff_works = np.mean(np.load(work_path+"/costsPartnered.npy"), axis=-1)

In [11]:
works, diff_works

(array([0.33697817, 0.48686108, 1.1457611 , 7.20046325]),
 array([0.        , 0.86028268, 1.4491079 , 8.37040459]))

In [12]:
SL_Ne = 50

In [13]:
SL_Ne2 = works[-1]*SL_Ne/works[-2]
SL_Ne2

314.22184146646293

In [14]:
SL_Ne3 = works[-1]*SL_Ne/works[-3]
SL_Ne3

739.4782204820567

In [15]:
SL_Ne4 = works[-1]*SL_Ne/works[-4]
SL_Ne4

1068.387201404896

In [16]:
from gpuocean.utils import MultiLevelAnalysis

In [17]:
analysis = MultiLevelAnalysis.Analysis(args_list, vars, diff_vars, works, diff_works)

In [18]:
ML_Ne = analysis.optimal_Ne(1.4e-1)
ML_Ne

array([98, 73, 60, 23], dtype=int32)

In [19]:
analysis.work(ML_Ne)/analysis.works[-1]

52.12029600358225

In [20]:
analysis2 = MultiLevelAnalysis.Analysis(args_list[1:], vars[1:], diff_vars[1:], works[1:], diff_works[1:])

In [21]:
ML_Ne2 = analysis2.optimal_Ne(1.3e-1)
ML_Ne2

array([143,  64,  24], dtype=int32)

In [22]:
analysis2.work(ML_Ne2)/analysis.works[-1]

50.44866384420629

SpeedUp

In [23]:
# works = [1/(8**2), 1/(8), 1]
# diff_works = [1/(8**2), 1/(8), 1]

In [24]:
# var_idx, tau = 0, 5e-2
# var_idx, tau = 1, 5e3
var_idx, tau = 2, 5e3

allwork = 0
for l_idx in range(len(ls)):
    if l_idx == 0: 
        allwork += np.sqrt(vars[l_idx][var_idx] * works[l_idx])
    else:
        allwork += np.sqrt(diff_vars[l_idx-1][var_idx] * diff_works[l_idx])

optNe_ref = np.zeros(len(ls))
for l_idx in range(len(ls)):
    if l_idx == 0: 
        optNe_ref[l_idx] = np.sqrt(vars[l_idx][var_idx]/works[l_idx]) * allwork
    else: 
        optNe_ref[l_idx] = np.sqrt(diff_vars[l_idx-1][var_idx]/diff_works[l_idx]) * allwork

Ne = np.int32(np.ceil(1/(tau**2)*optNe_ref))
print("Ne = ", Ne)

MLens_work = Ne[0]*works[0]
for l_idx in range(1, len(ls)):
    MLens_work += Ne[l_idx]*diff_works[l_idx]

SLens_work = vars[-1][var_idx]/(tau**2) * works[-1]

print("SpeedUp = ", SLens_work/MLens_work)

Ne =  [79 56 45 15]
SpeedUp =  1.2591642268708745
